In [1]:
import scanpy as sc
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder 
import torch

import os, sys
sys.path.append('./src')
from src.models.lib.data import *
datafile = 'data/mouse/Adult Inhibitory Neurons in Mouse.tsv'
labelfile = 'data/mouse/Adult Inhibitory Neurons in Mouse_labels.tsv'

label_df = pd.read_csv(labelfile, sep='\t')
label_df['numeric_class']

0         36
1         36
2          9
3         40
4         40
          ..
141064    36
141065    41
141066    41
141067    36
141068    36
Name: numeric_class, Length: 141069, dtype: int64

In [2]:
min(label_df['numeric_class'])

0

In [3]:
mouse_atlas = sc.read_h5ad('data/mouse/MouseAdultInhibitoryNeurons.h5ad')

In [4]:
mo_data = sc.read_h5ad('data/mouse/Mo_PV_paper_TDTomato_mouseonly.h5ad')
mo_data.var

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


,gene_ids,feature_types,genome
Xkr4,ENSMUSG00000051951,Gene Expression,mm10_tdTomato
Gm1992,ENSMUSG00000089699,Gene Expression,mm10_tdTomato
Gm19938,ENSMUSG00000102331,Gene Expression,mm10_tdTomato
Gm37381,ENSMUSG00000102343,Gene Expression,mm10_tdTomato
Rp1,ENSMUSG00000025900,Gene Expression,mm10_tdTomato
...,...,...,...
AC133095.2,ENSMUSG00000095475,Gene Expression,mm10_tdTomato
AC133095.1,ENSMUSG00000094855,Gene Expression,mm10_tdTomato
AC234645.1,ENSMUSG00000095019,Gene Expression,mm10_tdTomato
AC149090.1,ENSMUSG00000095041,Gene Expression,mm10_tdTomato


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(mouse_atlas.X, label_df['numeric_class'].values)

In [6]:
X_train[0].shape

(34430,)

In [10]:
atlas_train = NumpyStream(
    matrix=X_train,
    labels=y_train,
    sep='\t',
    columns=list(mouse_atlas.var.index),
)

In [11]:
atlas_val = NumpyStream(
    matrix = X_test,
    labels = y_test,
    sep='\t',
    columns=list(mouse_atlas.var.index),
    class_label='ingore'
)

In [12]:
len(atlas_val)

35268

In [13]:
train = DataLoader(atlas_train, batch_size=4, num_workers=0, drop_last=True, shuffle=True)
val = DataLoader(atlas_val, batch_size=4, num_workers=0, drop_last=True, shuffle=False)

In [ ]:
from src.models.lib.neural import GeneClassifier
from pytorch_lightning import Trainer 
from pytorch_lightning.loggers import WandbLogger

model = GeneClassifier(
    input_dim=34430, 
    output_dim=50,
    optim_params={
        'optimizer': torch.optim.Adam,
        'lr': 3e-4,
    }
)

wandb_logger = WandbLogger(project='Mouse Classifier', name='Tabnet with Metrics')
trainer = Trainer(logger=wandb_logger)

trainer.fit(model, train, val)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Model initialized. input_dim = 34430, output_dim = 50. Metrics are dict_keys(['accuracy', 'precision', 'recall']) and weighted_metrics = False


wandb: Currently logged in as: jlehrer1 (use `wandb login --relogin` to force relogin)



  | Name       | Type                 | Params
----------------------------------------------------
0 | base_model | TabNetGeneClassifier | 2.3 M 
----------------------------------------------------
2.3 M     Trainable params
0         Non-trainable params
2.3 M     Total params
9.114     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

wandb: Network error (ConnectionError), entering retry loop.


Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
def gene_intersection(
    files
):
    import dask.dataframe as dd 
    
    cols = []
    for file in files:
        temp = pd.read_csv(fpath, nrows=1, header=1).columns 
        temp = [x.split('|')[0].upper().strip() for x in temp]
        cols.append(set(temp))
    
    unique = list(set.intersection(*cols))
    unique = sorted(unique)
    
    return unique 